In [4]:
import re
from tqdm import tqdm
import gzip
import os
import regex
import numpy as np

#### Preparing of transliterated langs

In [5]:
def preparedata(lan, repeats, train=0):
    with gzip.open(f'./OSCAR-2109/packaged/{lan}/{lan}_part_1.txt.gz', 'rb') as f:
            text = f.readlines()
    type = 'test'
    for repeat in range(repeats):
        if repeat > repeats-train-1:
            type = 'train'
        file = open(f'./data/{lan}_{repeat}.{type}', 'w')
        i = 0
        skip = False
        for line in (text):
            if skip:
                break
            line = line.decode('utf-8')
            line = regex.sub(r'-[\p{L}_]', '', line)
            line = line[:-3].lower().replace(' ', '\x5f')
            line = [line[i:i+10] for i in range(0, len(line), 10)]
            for string in line:
                if len(string) == 10:
                    i += 1
                    if i < 100*(repeat):
                        continue
                    if i > 100*(repeat+1):
                        skip = True
                        break
                    file.write(f'{string}\n')

In [6]:
languages = ['ar', 'de', 'el', 'en', 'fr', 'hi', 'la', 'pl', 'ru', 'sw', 'zh']
for lan in tqdm(languages):
    preparedata(lan, 3)
preparedata('eo', 6, 3)

100%|██████████| 11/11 [01:24<00:00,  7.64s/it]


#### Preparing of transliterated langs

In [7]:
def preparetransl(lan, repeats, train=0):
    with open(f'./data/{lan}_transl.test', 'rb') as f:
            text = f.readlines()
    type = 'test'
    for repeat in range(repeats):
        if repeat > repeats-train-1:
            type = 'train'
        file = open(f'./data/{lan}_transl_{repeat}.{type}', 'w')
        i = 0
        skip = False
        for line in (text):
            if skip:
                break
            line = line.decode('utf-8')
            line = regex.sub(r'-[\p{L}_]', '', line)
            line = line[:-3].lower().replace(' ', '\x5f')
            line = [line[i:i+10] for i in range(0, len(line), 10)]
            for string in line:
                if len(string) == 10:
                    i += 1
                    if i < 100*(repeat):
                        continue
                    if i > 100*(repeat+1):
                        skip = True
                        break
                    file.write(f'{string}\n')

In [9]:
for lan in tqdm(['ar', 'el', 'hi', 'ru', 'zh']):
    preparetransl(lan, 3)

100%|██████████| 5/5 [00:00<00:00, 181.71it/s]


#### Make alphabets

In [10]:
def makealphabet():
    with open('./utf8_sequence_0-0x10ffff_assigned_including-unprintable-asis.txt', 'rb') as f:
        text = f.readlines()

    writefile = open('./alphabet.txt', 'w') 

    for line in text:
        line = line.decode('utf-8')
        line = [line[i:i+10] for i in range(0, len(line), 10)]
        for string in line:
            writefile.write(f'{string}\n')

makealphabet()

In [12]:
languages = ['ar', 'de', 'el', 'en', 'eo', 'fr', 'hi', 'la', 'pl', 'ru', 'sw'] #no chinese
transl = ['ar', 'el', 'hi', 'ru']

def makealphabet1(lan_set, transl=0): #lan_set: transl or not, if transl put transl=1
    gabet=np.array([])
    
    for lan in tqdm(lan_set):
        abet = np.array([])

        for r in range(3):
            if transl:
                with open(f'./data/{lan}_transl_{r}.test', 'r') as f:
                    text = f.readlines()
                    
            else:
                with open(f'./data/{lan}_{r}.test', 'r') as f:
                    text = f.readlines()

            with open(f'./data/eo_{r}.train', 'r') as f:
                eo = f.readlines()

            for line in text:
                chars = list(line)
                abet = np.append(abet, chars)
                abet = np.unique(abet)

            for line in eo:
                chars = list(line)
                abet = np.append(abet, chars)
                abet = np.unique(abet)

        gabet=np.append(gabet, abet)
        
        if transl:
            with open(f'./alphabets/alphabet_{lan}_transl.txt', 'w') as f:
                for char in abet:
                    f.write(char)
        else:
            with open(f'./alphabets/alphabet_{lan}.txt', 'w') as f:
                for char in abet:
                    f.write(char)
        
    return gabet

In [13]:
gabet = makealphabet1(languages) #literal
tgabet = makealphabet1(transl, 1) #transliteral
for ch in tgabet:
    gabet = np.append(gabet, ch) #combined (global)

100%|██████████| 4/4 [00:00<00:00,  4.57it/s]


In [14]:
c=0
with open('./alphabet.txt', 'w') as f:
    for char in np.unique(gabet):
        c+=1
        f.write(char)
print('Global alphabet length:', c)

Global alphabet length: 275
